<a href="https://colab.research.google.com/github/Sahadat31/QWEN3_HF_QUATIZE/blob/main/HF_QUANTIZE_MODELS_GWEN3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q --upgrade torch==2.5.1+cu124 torchvision==0.20.1+cu124 torchaudio==2.5.1+cu124 --index-url https://download.pytorch.org/whl/cu124
!pip install -q requests bitsandbytes==0.46.0 transformers accelerate==1.3.0

In [2]:
import torch
from google.colab import userdata
from huggingface_hub import login
from transformers import AutoTokenizer,AutoModelForCausalLM,TextStreamer,BitsAndBytesConfig
import gc

In [3]:
!git config --global credential.helper store
hf_token = userdata.get("HF_TOKEN")
login(hf_token,add_to_git_credential=True)

In [4]:
QWEN= "Qwen/Qwen3-8B"


In [5]:
messages=[
    {"role":"system","content":"You are an expert in data science"},
    {"role":"user","content":"How LLM's are different from traditional Machine Learning models?"}
]

In [6]:
#quantize weights precision to 4 bits floating number from 32 bits
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4"
)

In [7]:
tokenizer = AutoTokenizer.from_pretrained(QWEN)
tokenizer.pad_token = tokenizer.eos_token
inputs = tokenizer.apply_chat_template(messages,return_tensors="pt").to("cuda")

In [8]:
model = AutoModelForCausalLM.from_pretrained(QWEN,device_map="auto",quantization_config=quant_config)

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

model-00002-of-00005.safetensors:   0%|          | 0.00/3.99G [00:00<?, ?B/s]

model-00003-of-00005.safetensors:   0%|          | 0.00/3.96G [00:00<?, ?B/s]

model-00004-of-00005.safetensors:   0%|          | 0.00/3.19G [00:00<?, ?B/s]

model-00001-of-00005.safetensors:   0%|          | 0.00/4.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [9]:
memory = model.get_memory_footprint() / 1e6
print(f"Memory footprint: {memory:,.1f} MB")

Memory footprint: 5,962.8 MB


In [10]:
model

Qwen3ForCausalLM(
  (model): Qwen3Model(
    (embed_tokens): Embedding(151936, 4096)
    (layers): ModuleList(
      (0-35): 36 x Qwen3DecoderLayer(
        (self_attn): Qwen3Attention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (q_norm): Qwen3RMSNorm((128,), eps=1e-06)
          (k_norm): Qwen3RMSNorm((128,), eps=1e-06)
        )
        (mlp): Qwen3MLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=12288, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=12288, bias=False)
          (down_proj): Linear4bit(in_features=12288, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen3RMSNorm((4096,), eps=1e-06)
        

In [16]:
outputs = model.generate(inputs,max_new_tokens=100)
print(tokenizer.decode(outputs[0]))

<|im_start|>system
You are an expert in data science<|im_end|>
<|im_start|>user
How LLM's are different from traditional Machine Learning models?<|im_end|>
</think>

Large Language Models (LLMs) are a type of advanced machine learning model, but they differ significantly from traditional Machine Learning (ML) models in several key aspects. Here's a structured comparison:

---

### **1. Scale and Complexity**
- **LLMs**: 
  - Built on **transformer architectures** with **hundreds of billions of parameters**.
  - Trained on **massive datasets** (e.g., web text, books, code) spanning **terabytes


In [ ]:
'''
del inputs, outputs, model
torch.cuda.empty_cache()
'''